In [0]:
'''https://towardsdatascience.com/gan-by-example-using-keras-on-tensorflow-backend-1a6d515a60d0'''

'https://towardsdatascience.com/gan-by-example-using-keras-on-tensorflow-backend-1a6d515a60d0'

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, Flatten, Dense, Activation, LeakyReLU
from keras.layers import BatchNormalization, Reshape, UpSampling2D, Conv2DTranspose

In [0]:
class GAN():
  
  def __init__(self, input_shape):
    self.D = Sequential()
    self.construct_discriminator(input_shape)
    self.G = Sequential()
    self.construct_generator()
    
  def construct_discriminator(self,input_shape):  
    depth = 64
    dropout = 0.4
    # In: 28 x 28 x 1, depth = 1
    # Out: 14 x 14 x 1, depth=64
    #input_shape = (self.img_rows, self.img_cols, self.channel)
    self.D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,\
    padding='same', activation=LeakyReLU(alpha=0.2)))
    self.D.add(Dropout(dropout))
    self.D.add(Conv2D(depth*2, 5, strides=2, padding='same',\
    activation=LeakyReLU(alpha=0.2)))
    self.D.add(Dropout(dropout))
    self.D.add(Conv2D(depth*4, 5, strides=2, padding='same',\
    activation=LeakyReLU(alpha=0.2)))
    self.D.add(Dropout(dropout))
    self.D.add(Conv2D(depth*8, 5, strides=1, padding='same',\
    activation=LeakyReLU(alpha=0.2)))
    self.D.add(Dropout(dropout))
    # Out: 1-dim probability
    self.D.add(Flatten())
    self.D.add(Dense(1))
    self.D.add(Activation('sigmoid'))
    
  def construct_generator(self):
    dropout = 0.4
    depth = 64+64+64+64
    dim = 7
    # In: 100
    # Out: dim x dim x depth
    self.G.add(Dense(dim*dim*depth, input_dim=100))
    self.G.add(BatchNormalization(momentum=0.9))
    self.G.add(Activation('relu'))
    self.G.add(Reshape((dim, dim, depth)))
    self.G.add(Dropout(dropout))
    # In: dim x dim x depth
    # Out: 2*dim x 2*dim x depth/2
    self.G.add(UpSampling2D())
    self.G.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
    self.G.add(BatchNormalization(momentum=0.9))
    self.G.add(Activation('relu'))
    self.G.add(UpSampling2D())
    self.G.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
    self.G.add(BatchNormalization(momentum=0.9))
    self.G.add(Activation('relu'))
    self.G.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
    self.G.add(BatchNormalization(momentum=0.9))
    self.G.add(Activation('relu'))
    # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
    self.G.add(Conv2DTranspose(1, 5, padding='same'))
    self.G.add(Activation('sigmoid'))
    
    
  def summary(self):
    print("Discriminator: ")
    self.D.summary()
    print("Generator: ")
    self.G.summary()
    

In [0]:
nn1 = GAN((28,28,1))
nn1.summary()

/usr/local/lib/python3.6/dist-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Discriminator: 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 14, 14, 64)        1664      
_________________________________________________________________
dropout_17 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 7, 7, 128)         204928    
_________________________________________________________________
dropout_18 (Dropout)         (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 4, 4, 256)         819456    
_________________________________________________________________
dropout_19 (Dropout)         (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 4, 4, 512)         32773